In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
def load_dataset():
    df = pd.read_csv("/content/googleplaystore.csv")
    return df

In [ ]:
apps = load_dataset()
apps

In [ ]:
def print_summarize_df(dataset):
    print("This is information about dataset")
    print(dataset.info())
    print("This is statistic information about dataset")
    print(dataset.describe())
    print("This is shape of dataset")
    print(dataset.shape)
print_summarize_df(apps)

In [ ]:
def clean_category(row):
  row = row.replace("and", "&").replace("And", "&")
  return row

In [ ]:
def clean_installs(x):
    x = str(x)
    x = x.replace("+", "").replace(",", "").replace("Free", "0").replace(".", "")
    return int(x)

In [ ]:
def clean_price(x):
    x = str(x)
    x = x.replace("$", "0").replace("Everyone", "0")
    return float(x)

In [ ]:
def clean_genres(x):
    x = str(x)
    x = x.replace("&", "-").replace(";", " ")
    return x

In [ ]:
def clean_reviews(x):
    x = str(x)
    x = x.replace("M", "000000")
    return float(x)

In [ ]:
def clean_type(x):
    x = str(x)
    x = x.replace("nan", "Free").replace("0", "Free")
    return x

In [ ]:
def remove_M(x):
    return str(x).replace("M", "")

In [ ]:
def remove_plus(x):
    return str(x).replace("+", "")

In [ ]:
def remove_comma(x):
    return str(x).replace(",", "")

In [ ]:
def replace_varies(x):
    return "0" if x == "Varies with device" else x

In [ ]:
def clean_dataset(dataset):
    try:
        dataset.drop(['Current Ver', 'Android Ver', 'Last Updated'], axis=1, inplace=True)
    except:
        pass
    dataset["App"] = dataset["App"].str.title()
    dataset["Category"] = dataset["Category"].str.title().str.replace(r'[_]', ' ', regex=True)
    dataset["Category"] = dataset["Category"].apply(clean_category)
    dataset["Reviews"] = dataset["Reviews"].apply(clean_reviews)
    dataset["Type"] = dataset["Type"].apply(clean_type)
    dataset["Installs"] = dataset["Installs"].apply(clean_installs)
    dataset["Price"] = dataset["Price"].astype(str).apply(clean_price)
    dataset["Genres"] = dataset["Genres"].apply(clean_genres)
    dataset['Size'] = dataset['Size'].apply(remove_M)
    dataset['Size'] = dataset['Size'].apply(remove_plus)
    dataset['Size'] = dataset['Size'].apply(remove_comma)
    dataset['Size'] = dataset['Size'].apply(replace_varies)
    for i, size in dataset['Size'].items():
        if size.endswith("k"):
            dataset.loc[i, 'Size'] = float(size[:-1]) / 1024
        elif size.endswith("M"):
            dataset.loc[i, 'Size'] = float(size[:-1])
        else:
            dataset.loc[i, 'Size'] = float(size)
    dataset['Size'] = dataset['Size'].astype(int)        
    return dataset


In [ ]:
clean_dataset(apps)
cleaned_apps = clean_dataset(apps)

In [ ]:
cleaned_apps

In [ ]:
def print_summarize_dataset(dataset):
    print("This is information about dataset")
    print(dataset.info())
    print("This is statistic information about dataset")
    print(dataset.describe())
    print("This is shape of dataset")
    print(dataset.shape)
print_summarize_dataset(cleaned_apps)

In [ ]:
def print_histograms(dataset):
    bar= px.histogram(dataset, x = "Category", color = "Type",barmode = "group")
    bar.show()
print_histograms(cleaned_apps)    

In [ ]:
def compute_correlation_matrix(dataset):
  fig = px.imshow(dataset.select_dtypes(np.number).corr(),text_auto = True)
  fig.show()
compute_correlation_matrix(cleaned_apps)  

In [ ]:
def print_scatter_matrix():
    pd.plotting.scatter_matrix(cleaned_apps)
print_scatter_matrix()   

In [ ]:
family_category = cleaned_apps[(cleaned_apps['Category'] == "Family") & (cleaned_apps['Type'] != "Free")]
family_paid = family_category.sort_values(by="Installs", ascending=False).reset_index(drop=True)
top_10 = family_paid.head(10)
fig = px.bar(top_10, x='App', y='Installs', color='App', title='The most populars paid apps of Family category')
fig.show()

In [ ]:
fig = px.pie(top_10, names='Genres', values='Installs', color='Genres', title='The most popular genres according to the number of installations from paid family')
fig.show()

In [ ]:
installation_per_category = cleaned_apps.groupby(by="Category").sum()
installation_per_category_top_10 = installation_per_category.sort_values(by = "Installs", ascending = False).head(10)
figure = px.pie(installation_per_category_top_10, names = installation_per_category_top_10.index, values ='Installs', color = installation_per_category_top_10.index, title='The number of installation per category')
figure.show()

In [ ]:
installation_per_category_top_10

In [ ]:
mean_price=cleaned_apps.groupby('Category')['Price'].mean().dropna().reset_index().sort_values(by='Price',ascending=False)
fig = px.bar(mean_price, x=mean_price['Category'], y='Price', color=mean_price['Category'], title='The mean price per category')
fig.show()

In [ ]:
expensive_apps = cleaned_apps[cleaned_apps['Type'] != "Free"].sort_values(by='Price', ascending=False).head(20)
fig = px.bar(expensive_apps, x='App', y='Price', color='Category', title="Most expensive apps per category")
fig.show()